In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib import cm
from functools import partial

## Versuchsbeschreibung
Mit dem Farbrechner können Sie den Vorgang beim Umrechnen zwischen verschiedenen Farbräumen untersuchen. Probieren Sie verschiedene Farbwerte aus und beobachten Sie den Umrechnungsvorgang. 
Dieser Versuch soll Ihnen ein Gefühl für verschiedene Farbräume geben. Wählen Sie ein Bild aus und lassen Sie sich die verschiedenen Farbkomponenten dieses Bildes anzeigen.

In [2]:
# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}
image2plot = np.uint8(np.zeros([500, 500, 3]))
dict_colorspaces = {'rgb': [0, 0, 0], 'cmy': [0, 0, 0], 'yuv': [0, 0, 0], 'hsv': [0, 0, 0], 'cie': [0, 0]}
test_help = True

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()
    
def convert_cmy(image_in):
    
    for i in range(3):
        image_in[:, :, i] = 255 - image_in[:, :, i]
    
    return image_in

def colorspace2rgb(array_colorspace, input_colorspace = 'rgb'):
                                        
    if input_colorspace == 'rgb':
        return array_colorspace
    elif input_colorspace == 'cmy':
        return 255 - np.uint8(array_colorspace)
    elif input_colorspace == 'yuv':
        yuv2rgb = np.array([[1.0, 0.0, 1.13983], [1.0, -0.39465, -0.58060], [1.0, 2.03211, 0.0]])
        return np.uint8(np.abs(np.dot(yuv2rgb, array_colorspace)) * 255.0)
        #return np.squeeze(cv2.cvtColor(np.reshape(np.uint8(array_colorspace), (1, 1, 3)), cv2.COLOR_YUV2RGB))
    elif input_colorspace == 'hsv':
        raise NotImplementedError
    elif input_colorspace == 'cie':
        raise NotImplementedError
    
def rgb2colorspace(rgb_values, output_colorspace = 'rgb'):
    
    if output_colorspace == 'rgb':
        return rgb_values
    elif output_colorspace == 'cmy':
        return 255 - np.uint8(rgb_values)
    elif output_colorspace == 'yuv':
        rgb2yuv = np.array([[0.299, 0.587, 0.114], [-0.14713, -0.28886, 0.436], [0.615, -0.51499, -0.10001]])
        return np.dot(rgb2yuv, np.float32(rgb_values) / 255.0)
        #return np.squeeze(cv2.cvtColor(np.reshape(np.uint8(rgb_values), (1, 1, 3)), cv2.COLOR_RGB2YUV))
    elif output_colorspace == 'hsv':
        tmp = np.squeeze(cv2.cvtColor(np.reshape(np.uint8(rgb_values), (1, 1, 3)), cv2.COLOR_RGB2HSV))
        return np.array([tmp[0] / 255.0 * 360.0, tmp[1] / 255.0, tmp[2] / 255.0])
        #return tmp
    elif output_colorspace == 'cie':
        tmp = cv2.cvtColor(np.reshape(np.uint8(rgb_values), (1, 1, 3)), cv2.COLOR_RGB2XYZ)
        #return tmp
        if np.sum(tmp) > 0.0:
            return np.array([tmp[0, 0, 0] / np.sum(tmp), tmp[0, 0, 1] / np.sum(tmp)])
        else:
            return np.array([0.0, 0.0])
    
    
def create_color_box(name_colorspace = 'rgb'):
    
    global_layout = {'width': '200px'}
    global_label_layout = {'align-items': 'center'}

    def change_test(change):
        if change['type'] == 'change' and change['name'] == 'value':
            # detect which color space changed
            helper_string = change['owner'].description.split('.')
            string_colorspace = helper_string[0]
            char_colorspace = helper_string[1]
            index_list = list(string_colorspace).index(char_colorspace)
            
            dict_colorspaces[string_colorspace][index_list] = change['new']
            value_in_rgb = colorspace2rgb(dict_colorspaces[string_colorspace], string_colorspace)
            update_colors(value_in_rgb)
    
    colorspace_dict = {}
    
    if name_colorspace in ['rgb', 'cmy', 'yuv']:
        colorspace_dict['label'] = widgets.Label(name_colorspace.upper(), layout = global_label_layout)
        for char in list(name_colorspace):
            if name_colorspace == 'yuv':
                if char == 'y':
                    colorspace_dict[char] = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, disabled=True, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
                elif char == 'u':
                    colorspace_dict[char] = widgets.FloatSlider(value=0.0, min=-0.436, max=0.436, disabled=True, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
                elif char == 'v':
                    colorspace_dict[char] = widgets.FloatSlider(value=0.0, min=-0.615, max=0.615, disabled=True, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
            else:
                colorspace_dict[char] = widgets.IntSlider(value=255, min=0, max=255, disabled=False, continuous_update=False, orientation='horizontal',
                                                      readout=True, readout_format='d', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
                colorspace_dict[char].observe(change_test)
            
    elif name_colorspace == 'hsv':
        colorspace_dict['label'] = widgets.Label(name_colorspace.upper(), layout = global_label_layout)
        for char in list(name_colorspace):
            if char == 'h':
                colorspace_dict[char] = widgets.FloatSlider(value=120.0, min=0.0, max=360.0, step=0.05, disabled=True, continuous_update=False, orientation='horizontal',
                                                          readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
            else:
                colorspace_dict[char] = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.05, disabled=True, continuous_update=False, orientation='horizontal',
                                                          readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
    elif name_colorspace == 'cie':
        colorspace_dict['label'] = widgets.Label(name_colorspace.upper(), layout = global_label_layout)
        helper_colorspace = 'xy'
        for char in list(helper_colorspace):
            colorspace_dict[char] = widgets.FloatSlider(value=0.25, min=0.0, max=1.0, step=0.05, disabled=True, continuous_update=False, orientation='horizontal',
                                                          readout=True, readout_format='.2f', description = '%s.%s' % (name_colorspace, char), layout = global_layout)
        
    vbox = widgets.VBox(list(colorspace_dict.values()), layout = widgets.Layout(width = '100%', border='1px solid black', align_items = 'center'))
    
    return vbox
        

def update_colors(rgb_color_list):
    # use the rgb color and update everything
    for key, value in dict_colorspaces.items():
        dict_colorspaces[key] = rgb2colorspace(rgb_color_list, key)
                
    # set all slider
    list_color_spaces = ['rgb', 'cmy', 'yuv', 'hsv', 'cie']
    for name in list_color_spaces:
        if name != 'cie':
            for i in range(3):
                hbox.children[list_color_spaces.index(name)].children[i+1].value = dict_colorspaces[name][i]
        else:
            for i in range(2):
                hbox.children[list_color_spaces.index(name)].children[i+1].value = dict_colorspaces[name][i]
    
    # update color image
    for i in range(3):
        image2plot[:, :, i] = dict_colorspaces['rgb'][i]
                
    image_1.value = convert_image(image2plot)
              
        
# get chars for every colorspace 
names_colorspace = ['RGB', 'CMY', 'YUV', 'HSV', 'CIE']
chars_colorspace = [char for name in names_colorspace[:4] for char in list(name)]
# add X and Y instead of C I E
chars_colorspace.extend(['X', 'Y'])
chars_colorspace = chars_colorspace[::3] + chars_colorspace[1::3] + chars_colorspace[2::3]

items = [create_color_box(i.lower()) for i in names_colorspace]
hbox = widgets.HBox(items)

image_1 = widgets.Image(value=convert_image(image2plot), format='png', width=300, height=400)
label_1 = widgets.Label(value='Farbe')


box = widgets.VBox([label_1, image_1])

display(box)
display(hbox)